# Load all data and clean

We had two data 
- One gotten fromm the schools website (JSON file)
- One gotten from CITM (XLSX file)

In [29]:
import json
import pandas as pd
# Load your JSON data
with open('admission.json') as f:
    data = json.load(f)

# Excel file with extra data
df = pd.read_excel('programs.xlsx', sheet_name='Sheet1', header=0)

Here we made the first row the header of the file

In [32]:
df.columns = df.iloc[0]
df = df.drop(0)  # Remove the first row since it's now the header
df = df.reset_index(drop=True)
df.drop(columns=['sn'], inplace=True)

Here we filtered the dataset to take only extremly relevant colunms

In [35]:
df = df[['PCAcronym', 'PTAcronym', 'PNName', 'SchoolName']]
df.loc[:, "PTAcronym"] = df.PTAcronym.replace({
    "FT": "Full Time",
    "PT": "Part Time"
})
df = df[df['PTAcronym'] != 'FL']
df.sample(5)

,PCAcronym,PTAcronym,PNName,SchoolName
58,ND,Part Time,CIVIL ENGINEERING,ENGINEERING
72,ND,Full Time,FASHION DESIGN,"ARTS, DESIGN & PRINTING"
143,HND,Full Time,METALLURGICAL ENGINEERING,ENGINEERING
49,ND,Full Time,AGRICULTURAL TECHNOLOGY,TECHNOLOGY
152,ND,Full Time,MINERAL & PETROLEUM RESOURCES ENGINEERING,ENGINEERING


I have not heard of these certifications before so I removed it abeg

In [38]:
v_t_r = ['CP', 'ADV CERT', 'TECH CERT', 'PGD']
df_cleaned = df[~df['PCAcronym'].isin(v_t_r)]

# Function to convert text to sentence case
def to_sentence_case(text):
    if isinstance(text, str):
        return text.capitalize()
    return text

# Convert selected columns to sentence case
columns_to_convert = ['PNName', 'SchoolName']  # Replace with your column names
df_cleaned.loc[:, columns_to_convert] = df_cleaned.loc[:, columns_to_convert].apply(lambda col: col.map(to_sentence_case))

df_cleaned.sample(5)

C:\Users\hhhhhh\AppData\Local\Temp\ipykernel_17668\290486379.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.loc[:, columns_to_convert] = df_cleaned.loc[:, columns_to_convert].apply(lambda col: col.map(to_sentence_case))


,PCAcronym,PTAcronym,PNName,SchoolName
238,HND,Part Time,Film & multimedia studies,Liberal studies
191,HND,Full Time,Mechanical (manufacturing) engineering,Engineering
159,B.SC(ED),Part Time,Art education,Technical education
156,HND,Part Time,Computer engineering,Engineering
169,B.SC(ED),Part Time,Art education 200level,Technical education


# Create filters like a pivot table

Everything I am doing here is filtering each of the elements of admission so it would be easy to loop through and create a knowledge base

In [42]:
# All HND programs
hnd_programs = df_cleaned[df_cleaned["PCAcronym"] == "HND"]
grouped_hnd_school = hnd_programs.groupby("SchoolName")["PNName"].apply(list).reset_index()
grouped_hnd_p_type = hnd_programs.groupby("PTAcronym")["PNName"].apply(list).reset_index()

In [43]:
# All ND programs
nd_programs = df_cleaned[df_cleaned["PCAcronym"] == "ND"]
grouped_nd_school = nd_programs.groupby("SchoolName")["PNName"].apply(list).reset_index()
grouped_nd_p_type = nd_programs.groupby("PTAcronym")["PNName"].apply(list).reset_index()

In [44]:
# All B.SC(ED) programs
ed_programs = df_cleaned[df_cleaned["PCAcronym"] == "B.SC(ED)"]
grouped_ed_school = ed_programs.groupby("SchoolName")["PNName"].apply(list).reset_index()
grouped_ed_p_type = ed_programs.groupby("PTAcronym")["PNName"].apply(list).reset_index()

In [48]:
# All full time programs
full_time_programs = df_cleaned[df_cleaned["PTAcronym"] == "Full Time"]
grouped_ft_school = full_time_programs.groupby("SchoolName")["PNName"].apply(list).reset_index()

# all part time programs
part_time_programs = df_cleaned[df_cleaned["PTAcronym"] == "Part Time"]
grouped_pt_school = full_time_programs.groupby("SchoolName")["PNName"].apply(list).reset_index()

In [50]:
grouped_hnd_p_type

,PTAcronym,PNName
0,Full Time,"[Accountancy, Architecture, Biochemistry, Book..."
1,Part Time,"[Accountancy, Building technology, Business ad..."


# Creating the questions
Here the questions and paragraphs for the knowledge base was created.

In [55]:
# Questions from the JSON file 
question_answer_pairs = []

for item in data["AdmissionRequirements"]:
    for dept, requirements in item.items():
        dept = dept,
        o_level_com = ", ".join(requirements["OLEVEL_COMPULSORY"]),
        o_level_elect = ", ".join(requirements["OLEVEL_ELECTIVE"]),
        utme =  ", ".join(requirements["UTME_REQUIREMENT"])


        print("Department -", dept, "\nO'level compulsory -", o_level_com, "\nO'level Elective -", o_level_elect, "\nUTME -", utme, '\n')
        
    #     question_answer_pairs.append({
    #     "answer": f"""Admission to the {str(dept).strip('(),').lower()} program requires a minimum UTME/JAMB score of 180. 
    #     The UTME/JAMB subject combination includes:
    #     {str(utme).strip('(),').lower()}
        
    #     The compulsory O-Level subjects for {str(dept).strip('(),').lower()} are:
    #     {str(o_level_com).strip('(),').lower()}
        
    #     Elective O-Level subjects for {str(dept).strip('(),').lower()} include:
    #     {str(o_level_elect).strip('(),').lower()}

    #     All required subjects for {str(dept).strip('(),').lower()} are:
    #     Compulsory: {str(o_level_com).strip('(),').lower()}, Elective: {str(o_level_elect).strip('(),').lower()}
        
    #     Candidates must have at least five credits, including Mathematics and English Language.
        
    #     The admission process for the {str(dept).strip('(),').lower()} program involves the following steps:
    #     1. Obtain a minimum UTME/JAMB score of 180.
    #     2. Ensure you have the required O-Level subjects.
    #     3. Participate in YABATECH's post-UTME screening exercise.
    #     4. Check the admission list on the school's portal.
    #     """
    # })

        
    #     question_answer_pairs.append({
    #     "Q": f"What are the UTME/JAMB subject combination for {str(dept).strip('(),').lower()}?, answer: {str(utme).strip('(),').lower()}"
    # })
    #     question_answer_pairs.append({
    #     "Q": f"What are the compulsory O-Level subjects for {str(dept).strip('(),').lower()}?, answer: {str(o_level_com).strip('(),').lower()}"
    # })
    #     question_answer_pairs.append({
    #     "Q": f"What are the elective O-Level subjects for {str(dept).strip('(),').lower()}?, answer: {str(o_level_elect).strip('(),').lower()}"
    # })

    #     question_answer_pairs.append({
    #     "Q": f"What is the UTME/JAMB cutoff mark for {str(dept).strip('(),').lower()}, answer: 180"
    # })

    #     question_answer_pairs.append({
    #     "Q": f"""List all required subjects for {str(dept).strip('(),').lower()}., answer: Compulsory: {str(o_level_com).strip('(),').lower()}, Elective: {str(o_level_elect).strip('(),').lower()}"""
    # })

Department - ('ACCOUNTANCY',) 
O'level compulsory - ('Mathematics, English Language, Economics',) 
O'level Elective - ('Commerce, Geography, Biology/Agricultural Science, Government/Civic Education, Business Management/Method and Financial Accounting, Literature in English',) 
UTME - English Language, Mathematics, Geography, Commerce, Government/History, Principle of Accounts, Economics 

Department - ('AGRICULTURAL & BIO-ENVIRONMENTAL ENGINEERING',) 
O'level compulsory - ('Mathematics, English Language, Physics, Chemistry, Biology/Agricultural Science',) 
O'level Elective - ('',) 
UTME - Mathematics, English Language, Physics, Chemistry 

Department - ('AGRICULTURAL TECHNOLOGY',) 
O'level compulsory - ('Mathematics, English Language, Chemistry, Biology/Agric Science',) 
O'level Elective - ('Physics, Geography, Economics, Technical Drawing, Food & Nutrition, Animal Husbandry and Fisheries, Civic Education',) 
UTME - English Language, Chemistry, Biology/Agric Science, Mathematics, Geogr

In [48]:
for index, row in df_cleaned.iterrows():
    program = row["PNName"]
    program_acronym = row["PCAcronym"]
    program_type = row["PTAcronym"]
    school = "School of " + row["SchoolName"]

    question_answer_pairs.append({
        "answer": f"""The {program} is offered by the {school}, this program is available as a {program_type}"""
    })

    question_answer_pairs.append({
        "Q": f"Which school offers the {program} program?, answer: The {program} program is offered by the {school}"
    })

    question_answer_pairs.append({
        "Q": f"Is {program} available as a {program_type} program?, answer: Yes, it is available as a {program_type} program."
    })

In [49]:
# All full time courses
ft_courses = []
for index, row in full_time_programs.iterrows():
    program = row["PNName"]
    program_acronym = row["PCAcronym"]
    program_type = row["PTAcronym"]
    school = "School of " + row["SchoolName"]
    ft_courses.append(program)

# All part time courses
pt_courses = []
for index, row in full_time_programs.iterrows():
    program = row["PNName"]
    program_acronym = row["PCAcronym"]
    program_type = row["PTAcronym"]
    school = "School of " + row["SchoolName"]
    pt_courses.append(program)

# HND requirements answer
for index, row in hnd_programs.iterrows():
    program_type = row["PTAcronym"]
    program = row["PNName"]
    question_answer_pairs.append({
        "Q": f"""What are the HND requirements for {program}, answer: The HND requiremets for {program} are - One year mandatory Industrial attachment - OND certification from an accredited institution - O'level and UTME combination used for OND"""
    })

    question_answer_pairs.append({
        "answer": f"""The HND requirements for {program} include a one-year mandatory industrial attachment, an OND certification from an accredited institution, and the O'level and UTME combination that was used for the OND program."""
    })

# B.SC(ED)
for index, row in grouped_ed_school.iterrows():
    school = row["SchoolName"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What B.SC(ED) programs are offered by the School of {school}?, answer: The School of {school} offers the following B.Ed programs: {programs}."""
    })

    question_answer_pairs.append({
        "answer": f"""The School of {school} offers the following B.Ed programs: {programs}."""
    })

# Generate questions and answers for programs by program type (education)
for index, row in grouped_ed_p_type.iterrows():
    program_type = row["PTAcronym"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What B.Ed programs are available as {program_type} programs?, answer: The following B.SC(ED) programs are available as {program_type} programs: {programs}."""
    })

    question_answer_pairs.append({
        "answer": f"""The following B.Ed programs are available as {program_type} programs: {programs}."""
    })


# Answers for HND (school offering each course)
for index, row in grouped_hnd_school.iterrows():
    school = row["SchoolName"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What HND programs are offered by the School of {school}?, answer: The School of {school} offers the following HND programs: {programs}."""
    })

    question_answer_pairs.append({
        "Q": f"""The School of {school} offers the following HND programs: {programs}.."""
    })
    

# Generate questions and answers for programs by program type (HND)
for index, row in grouped_hnd_p_type.iterrows():
    program_type = row["PTAcronym"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What HND programs are available as {program_type} programs?, answer: The following HND programs are available as {program_type} programs: {programs}."""
    })

# Answers for OND (school offering each course)
for index, row in grouped_nd_school.iterrows():
    school = row["SchoolName"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What ND programs are offered by the School of {school}?, answer: The School of {school} offers the following ND programs: {programs}."""
    })

# Generate questions and answers for programs by program type (OND)
for index, row in grouped_nd_p_type.iterrows():
    program_type = row["PTAcronym"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What ND programs are available as {program_type} programs?, answer: The following ND programs are available as {program_type} programs: {programs}."""
    })

# Full time programs offered by each school
for index, row in grouped_ft_school.iterrows():
    school = row["SchoolName"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What programs are offered by the School of {school} as a full time program? answer: The School of {school} offers the following programs: {programs} as full time."""
        
    })

# part time programs offered by each school
for index, row in grouped_ft_school.iterrows():
    school = row["SchoolName"]
    programs = ", ".join(row["PNName"])
    question_answer_pairs.append({
        "Q": f"""What programs are offered by the School of {school} as a part time program?, answer: The School of {school} offers the following programs: {programs} as part time."""
    })

In [52]:
#  Other questions
question_answer_pairs.append({
        "Q": "Where is YABATECH located? answer: Yaba College of Technology, Herbert Macaulay Road, Opposite WAEC office, Yaba Lagos."
    })

question_answer_pairs.append({
        "Q": "HND requirements for all courses answer: One year mandatory Industrial Attachment"   
    })

question_answer_pairs.append({
        "Q": f"What are all the available full-time courses or programs offered?, answer: The following are the list of all full time courses: {ft_courses}"  
    })

question_answer_pairs.append({
        "Q": f"What are all the available part-time courses or programs offered, answer: The following are the list of all full time courses: {pt_courses}"   
    })

question_answer_pairs.append({
        "Q": """Could you provide a list of all the available program types?, answer: The available program types include: 
        - Ordinary National Diploma (OND)
        - Higher National Diploma (HND)
        - Bachelor of Education (B.Ed)"""
})


Here we were able to see how many FAQs we generated

In [54]:
len(question_answer_pairs)

1113

# Creating paragraphs about each department

In [25]:
question_answer_pairs

[{'answer': "Admission to the 'accountancy' program requires a minimum UTME/JAMB score of 180. \n        The UTME/JAMB subject combination includes:\n        english language, mathematics, geography, commerce, government/history, principle of accounts, economics\n        \n        The compulsory O-Level subjects for 'accountancy' are:\n        'mathematics, english language, economics'\n        \n        Elective O-Level subjects for 'accountancy' include:\n        'commerce, geography, biology/agricultural science, government/civic education, business management/method and financial accounting, literature in english'\n\n        All required subjects for 'accountancy' are:\n        Compulsory: 'mathematics, english language, economics', Elective: 'commerce, geography, biology/agricultural science, government/civic education, business management/method and financial accounting, literature in english'\n        \n        Candidates must have at least five credits, including Mathematics an

In [26]:
# # Save as a JSON file
# with open("programs.json", "w") as json_file:
#     json.dump(question_answer_pairs, json_file, indent=4)

In [56]:
# Save as a text file
with open("program.txt", "w") as txt_file:
    for item in question_answer_pairs:
        for key, value in item.items():
            txt_file.write(f"{key}: {value}\n")
        txt_file.write("\n")  # Add a newline between each dictionary